# Upscale ASO Data from 3m to 9m Resolution
## For easier plotting

In [5]:
!pip install earthpy

In [1]:
import os
import urllib
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import numpy as np
import earthpy as et

In [2]:
import glob
from osgeo import gdal
import rasterio as rio
import rasterio.plot
from rasterio.plot import plotting_extent
from rasterio.enums import Resampling

In [3]:
#set working directory?
os.chdir(os.path.join(et.io.HOME, 
                      'ICESat-2_Snowdepth'))

In [4]:
dem_path = os.path.join("data",
                       "mcc_dem_3p0m_agg_TUOtrimmed_MANUAL.tif")
output_fn = os.path.join('data',
                        'mcc_dem_3p0m_agg_TUOtrimmed_MANUAL_9m.tif')

In [5]:
upscale_factor = 1/3 # ooopsies isn't this technically a downscale? my bad ---eric

with rasterio.open(dem_path) as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.nearest #billinear resamping..... change here
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

In [6]:
dem_9m = data[0,:,:]

In [7]:
with rasterio.open(dem_path) as src:
    profile = src.profile.copy()

    aff = src.transform
    newaff = rasterio.Affine(aff.a / upscale_factor, aff.b, aff.c,
                             aff.d, aff.e / upscale_factor, aff.f)
    profile.update({
            'dtype': 'float32',
            'height': dem_9m.shape[0],
            'width': dem_9m.shape[1],
            'transform': newaff})  

    with rasterio.open(output_fn, 'w', **profile) as dst:
        dst.write_band(1, dem_9m)